In [ ]:
# default_exp ldhdata
# export
import os
from collections import defaultdict
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import pytorch_lightning as lit
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from pathlib import Path

from nltk.tokenize import sent_tokenize
from sklearn.model_selection import GroupKFold, train_test_split
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

In [ ]:
# export
class LDHData:
    def __init__(self, data_dir):
        """Initializes the directories from which the data will be coming and leaving."""
        input_dir = Path(data_dir, "data")
        output_dir = Path(data_dir, "output")
        self.train_dir = (
            input_dir / "training"
        )  # Where the training data is coming from
        self.eval_dir = input_dir / "eval"  # Test data
        # TODO: assert that the directories are valid.
        self.datasets = defaultdict(str)
        self.save_dir = output_dir  # Where cached data is saved.

    @property
    def train_dataset(self):
        return self.datasets["train"]

    @property
    def eval_dataset(self):
        return self.datasets["eval"]

    def load_training_data(self, force_reload=False, save_datasets=True) -> None:
        training_save_dir = self.save_dir / "training"
        try:
            if force_reload:
                raise Exception()
            # If the training data has already been save, load it from the save_directory
            self.datasets["train"] = DatasetDict.load_from_disk(training_save_dir)
            print("Training data loaded from disk.")
        except:
            # If it hasn't regenerate the training data.
            print("Regenerating training data.")
            train_df_dict = self._parse_training_data(self.train_dir)
            self.datasets["train"] = DatasetDict(
                {
                    "far": Dataset.from_pandas(train_df_dict["far"]),
                    "obj": Dataset.from_pandas(train_df_dict["obj"]),
                }
            )
            if save_datasets:
                print(f"Saving training dataset to {training_save_dir}")
                self.datasets["train"].save_to_disk(training_save_dir)

    def load_eval_data(self, force_reload=False, save_datasets=True) -> None:
        eval_save_dir = self.save_dir / "eval"
        try:
            if force_reload:
                raise Exception()
            self.datasets["eval"] = DatasetDict.load_from_disk(eval_save_dir)
            print("Evaluation data loaded from disk.")
        except:
            print("Regenerating evaluation data.")
            eval_df_dict = self._parse_eval_data(self.eval_dir)
            self.datasets["eval"] = DatasetDict(
                {
                    "far": Dataset.from_pandas(eval_df_dict["far"]),
                    "obj": Dataset.from_pandas(eval_df_dict["obj"]),
                }
            )
            if save_datasets:
                print(f"Saving evaluation dataset to {eval_save_dir}")
                self.datasets["eval"].save_to_disk(eval_save_dir)

    def collapse_eval_data(self, df: pd.DataFrame):
        """Let df be the dataframe obtained from loading evaluation data.
        Expand the text in 'response' to have a single sentence per response.
        source: https://medium.com/@johnadungan/expanding-lists-in-panda-dataframes-2724803498f8
        """
        df["response"] = df["response"].map(sent_tokenize, na_action="ignore")
        texts = df["response"].dropna()
        lens_of_lists = texts.apply(len)
        origin_rows = range(texts.shape[0])
        destination_rows = np.repeat(origin_rows, lens_of_lists)
        non_list_cols = [idx for idx, col in enumerate(df.columns) if col != "response"]
        expanded_df = df.iloc[destination_rows, non_list_cols].copy()
        expanded_df["split_response"] = [i for items in texts for i in items]
        expanded_df = expanded_df[expanded_df["split_response"] != "."].reset_index(
            drop=True
        )
        expanded_df.rename(columns={"split_response": "response"}, inplace=True)
        return expanded_df

    # Functions to read the data files directly
    def _parse_training_data(self, train_data_dir: str) -> Dict[str, pd.DataFrame]:
        study1 = pd.read_csv(Path(train_data_dir, "Master_Final_TrainingData.csv"), usecols = ['Text Response', "AVG_OBJ", "AVG_FAR"])
        study1far = study1.rename(columns={
            'Text Response': 'response',
            'AVG_FAR': 'score'
        }).drop(columns='AVG_OBJ')

        study1obj = study1.rename(columns={
            'Text Response': 'response',
            'AVG_OBJ': 'score'
        }).drop(columns='AVG_FAR')


        return {
            'far': study1far,
            'obj': study1obj
        }

    def _parse_eval_data(self, eval_data_dir: str) -> Dict[str, pd.DataFrame]:
        """ Parses evaluation data.

        Args:
            eval_dir (str): The folder in which original training data is stored.

        Returns:
            Dict[str, pd.DataFrame]: [description]
        """
        # Read the excel files
        eval_far_data = pd.read_excel(
            os.path.join(eval_data_dir, "Alg_Far_NEW.xlsx"), engine="openpyxl"
        ).rename(columns={"TextResponse": "response"})
        eval_obj_data = pd.read_excel(
            os.path.join(eval_data_dir, "Alg_Obj_NEW.xlsx"), engine="openpyxl"
        ).rename(columns={"TextResponse": "response"})
        eval_far_data = eval_far_data[eval_far_data["response"].notna()]
        eval_obj_data = eval_obj_data[eval_obj_data["response"].notna()]
        return {
            "far": self.collapse_eval_data(eval_far_data),
            "obj": self.collapse_eval_data(eval_obj_data),
        }


def _expand_response(input_response: str) -> List[str]:
    sentences = sent_tokenize(input_response)
    return sentences

In [ ]:
# export
DEFAULT_TOKENIZER = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", use_fast=True
)

class LDHDataModule(lit.LightningDataModule):
    def __init__(
        self,
        data_dir,
        batch_size=16,
        tokenizer=DEFAULT_TOKENIZER,
        strat="obj",
        kfolds=1,
    ):
        super().__init__()
        self.data = LDHData(data_dir)
        self.strat = strat
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.kfolds = kfolds
        self.current_split = 0
        self.train_data = None
        self.eval_data = None
        self.splits = []

        # self.load_training_data(data, force_reload)
        # self.load_eval_data(data, force_reload)
        # self.splits = self.generate_splits(self.kfolds)

    def get_train_data(self, encoded=False):
        if not self.train_data:
            raise Exception("Please load the training data first!")

        train_data = self.train_data[self.strat]
        if not encoded:
            return train_data
        print("Encoding Training Data:")
        encoded_ds = train_data.map(
            lambda ds: self.tokenizer(
                ds["response"],
                add_special_tokens=True,
                padding="max_length",
                max_length=150,
            )
        )
        encoded_ds.set_format(
            type="torch",
            columns=["input_ids", "attention_mask", "score"],
        )
        return encoded_ds

    def load_train_data(self, force_reload=False):
        self.data.load_training_data(force_reload)
        train_data: Dataset = self.data.train_dataset[self.strat]
        print("Encoding Training Data:")
        encoded_ds = train_data.map(
            lambda ds: self.tokenizer(
                ds["response"],
                add_special_tokens=True,
                padding="max_length",
                max_length=150,
            )
        )
        encoded_ds.set_format(
            type="torch",
            columns=["input_ids", "attention_mask", "score"],
        )
        self.train_data = encoded_ds
        self.splits = self.generate_splits(self.kfolds)


    def load_eval_data(self, force_reload=False):
        self.data.load_eval_data(force_reload)
        eval_data: Dataset = self.data.eval_dataset[self.strat]
        print("Encoding Test Data")
        encoded_ds = eval_data.map(
            lambda ds: self.tokenizer(
                ds["response"],
                add_special_tokens=True,
                padding="max_length",
                max_length=150,
            )
        )
        # Need to include addcode here as well
        encoded_ds.set_format(
            type="torch",
            columns=["input_ids", "attention_mask"],
        )
        self.eval_data = encoded_ds

    def generate_splits(self, num_groups=1)-> List[Tuple[int, int]]:
        """Generates splits of training data.
        Args:
            num_groups (int, optional): [description]. Defaults to 1.

        Returns:
            List[Tuple[int, int]]: [description]
        """
        train_data = self.train_data
        indices = np.arange(len(train_data))
        if num_groups == 1:
            return [train_test_split(indices, train_size=0.1)]
        indices = indices % num_groups
        np.random.shuffle(indices)
        cv = GroupKFold(num_groups)
        splits = list(cv.split(train_data, groups=indices))
        return splits

    def get_train_dataloader(self, split: int):
        train_idx = self.splits[split][0].tolist()  # retrieve the split generated by GroupKFold
        data = Subset(self.train_data, train_idx)
        return DataLoader(data, batch_size=self.batch_size)

    def get_val_dataloader(self, split: int):
        val_idx = self.splits[split][1].tolist()  # retrieve the split generated by GroupKFold
        data = Subset(self.train_data, val_idx)
        return DataLoader(data, batch_size=self.batch_size)

    def train_dataloader(self):
        return self.get_train_dataloader(self.current_split)

    def val_dataloader(self):
        return self.get_val_dataloader(self.current_split)

    # def test_dataloader(self):
    #     return DataLoader(self.eval_data, batch_size=self.batch_size)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script("LDHData.ipynb")


Converted LDHData.ipynb.


In [ ]:
data = LDHDataModule(data_dir='/Users/danielpham/Google Drive/ldh/')
data.load_train_data()
data.load_eval_data()

data.get_train_data()
data.get_eval_data()


Training data loaded from disk.
Evaluation data loaded from disk.


Dataset({
    features: ['Bin', 'Condition', 'Cue1', 'Cue2', 'Group', 'Group_ID', 'Rating', 'ReappSuccess', 'Subj_ID', 'addcode', 'daycode', 'observed', 'response'],
    num_rows: 32109
})